# RoboMaster AI Challenge - YOLO Trainer
Armour localisation/identification with YOLO (You Only Look Once: one-shot object detection algorithm)
> Framework: darknet https://github.com/AlexeyAB/darknet

> Architecture:
* YOLOv3, YOLOv4 - original model focus on superior performance
* YOLOv3-tiny, YOLOv4-tiny - lightweight model suitable for mobile application 

> Requirements:
* Internet connection
* Google Drive with at least 500MB free space (need more if you are training with non-tiny models)

##COMP90082 - Software Project - RM-koala

Team member (alphabetical order):

* Akhtar Kurniawan (Akhtar)
* Che-Hao Chang (Ryan)
* Isaac Pedroza (Isaac)
* Jia Yin (Jia)
* Sejin Kim (Kim)


##System setup/checking

####Mount your Google Drive

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

####Verify GPU and CUDA driver

In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
import psutil
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]
process = psutil.Process(os.getpid())

print("GPU RAM {}MB".format(gpu.memoryTotal))

!/usr/local/cuda/bin/nvcc --version

##File retrieving/uploading

####Clone darknet repo & Configure the runtime settings & Make the executable

In [ ]:
%cd content/gdrive/My\ Drive
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make
!chmod +x ./darknet

###Define auxiliary functions

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)

###File type overview
> Download online
* .conv: Initial weight file

> Create your own
* .cfg: Configuration file, defines the neural network architecture
* .data: Points to the data list txt file
* .txt: Contains list of paths to train/ test/ valid images
* .names: Defines the class label strings

> Training/ Validation/ Testing dataset
* Images: JPG/PNG supported
* Labels: Put in the same directory and with the same name as the corresponding image, but with .txt-extension.
```
Label format: <object-class> <x> <y> <width> <height>
```

####Retrieve initial weight files

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
!wget https://pjreddie.com/media/files/darknet53.conv.74
!wget https://drive.google.com/file/d/18v36esoXCh-PsOKwyP2GWrpYDptDY8Zf/view?usp=sharing

###Use our examples

In [ ]:
# loader files
%cd data
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/file_list/rm_armour_train.txt
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/file_list/rm_pose_train.txt

# image data
!mkdir images
%cd images
!mkdir rm_koala
%cd rm_koala
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/images/armour200each.zip
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/images/pose200each.zip
!unzip armour200each.zip
!unzip pose200each.zip
%cd ../../..

# configuration files
%cd cfg
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/cfg/yolov4-tiny_2class_mod.cfg
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/cfg/yolov4-tiny_3class.cfg
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/cfg/rm_armour.data
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/cfg/rm_armour.names
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/cfg/rm_pose.data
!wget https://github.com/cchia790411/rm_ai_challenge_2020s2_koala/raw/master/src/cfg/rm_pose.names
%cd ..

###Upload your own

####Upload your .cfg/ .data/ .name files

In [ ]:
%cd cfg
upload()
%cd ..

####Upload your image list

In [ ]:
%cd data
upload()
%cd ..

####Upload your dataset
> We recommend you compress them all into a zip file and unzip after upload

In [ ]:
%cd data
!mkdir images
%cd images
upload()
!unzip YOUR_ZIPPED_DATASET.zip
%cd ../..

##Training/Verifying

####Start training

Change the capital part to your file name and run it to start training
```
!./darknet detector train cfg/YOUR_DATA.data cfg/YOUR_DATA.cfg INITIAL_WEIGHT -dont_show
```

In [ ]:
## Example
!./darknet detector train cfg/rm_pose.data cfg/yolov4-tiny_3class.cfg yolov4-tiny.conv.29 -dont_show

####Pick up training in case of the training is interrupted

Change the capital part to your file name and run it to resume training
```
!./darknet detector train cfg/YOUR_DATA.data cfg/YOUR_CFG.cfg backup/YOUR_CFG_last.weights -dont_show
```

In [ ]:
## Example
!./darknet detector train cfg/rm_pose.data cfg/yolov4-tiny_3class.cfg backup/yolov4-tiny_3class_last.weights -dont_show

####Verify training result

Change the capital part to your file name and run it to resume training
```
!./darknet detector test cfg/YOUR_DATA.data cfg/YOUR_CFG.cfg backup/YOUR_CFG_last.weights PATH_TO_TEST_IMAGE -dont_show
```

###darknet framework command overview
>Command format
* !./darknet detector test [path to .data file] [path to config] [path to weights] [path to image]

>Command line flags
* -ext_output: output bounding box location
* -thresh: Set confidence threshold for prediction
* -out: Output result as json file
* -map: Show mean average percision over training when there is a validation data set 
* -dont_show: Don't show prediction result in place


In [ ]:
## Example
!./darknet detector test cfg/rm_pose.data cfg/yolov4-tiny_3class.cfg backup/yolov4-tiny_3class_last.weights data/images/rm_koala/pose200each/blue_2_frame0060.jpg -ext_output -dont_show -thresh 0.5
imShow('predictions.jpg')